In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Phase diagram of the Hubbard model with Mean Field Theory

More details on the formalism [here](https://arxiv.org/pdf/1403.2259.pdf).

We place ourselves at half filling ($\mu = \frac U 2$) and look for antiferromagnetic solutions ($m_{i,1} = m_{i,2}$) assuming spatial homogeneity ($\langle n_{i,\alpha,\sigma} \rangle = \langle n_{\alpha,\sigma} \rangle$).

$\langle n_{\alpha, \sigma}\rangle$ is encoded in a vector:
\begin{pmatrix}
\langle n_{1, \uparrow}\rangle \\
\langle n_{1, \downarrow}\rangle \\
\langle n_{2, \uparrow}\rangle \\
\langle n_{2, \downarrow}\rangle \\
\end{pmatrix}

Among the 4 $\langle n_{\alpha, \sigma}\rangle$, only one is independant because of the assumptions. For $\alpha = 1,2$ we define:

\begin{cases}
\langle n_\alpha \rangle = \langle n_{\alpha, \uparrow} \rangle + \langle n_{\alpha, \downarrow} \rangle \\
\langle m_\alpha \rangle = \langle n_{\alpha, \uparrow} \rangle - \langle n_{\alpha, \downarrow} \rangle \\
\end{cases}

Then the assumptions lead to:

\begin{cases}
\langle n_1 \rangle = \langle n_2 \rangle = 1 \\
\langle m_1 \rangle = - \langle m_2 \rangle \\
\end{cases}

Therefore, the self-consistency equation can be written simply as $\langle m_1 \rangle = f \left( \langle m_1 \rangle \right)$. In practice, we compute $\langle n_{\alpha,\sigma} \rangle$ from $\langle m_1 \rangle$ with those constraints, and then use it to return $\langle m_1 \rangle$.


### $\vec k$ vectors

With the chosen unit cell, the first Brillouin zone is defined by $||\vec k||_1 < \frac \pi a$

$\vec k$ vectors are denoted by integers $(h,l)$ such that $\vec k = \frac \pi L (h \vec e_x + l \vec e_y)$ where $\vec e_x$ and $\vec e_y$ are unit vectors along $\vec b_1$ and $\vec b_2$ respectively. They are encoded by a vector of size 2 with components $h$ and $l$. With this notation, the first brillouin zone corresponds to integers $(h,l)$ verifying $|h| < \sqrt \frac N 2$ and $|l| < \sqrt \frac N 2$.

Since $\vec b_1 = \vec a_1 - \vec a_2$ and $\vec b_2 = \vec a_1 + \vec a_2$, we have:

\begin{cases}
\vec k \cdot \vec b_1 = \pi \sqrt{\frac 2 N} h \\
\vec k \cdot \vec b_2 = \pi \sqrt{\frac 2 N} l
\end{cases}

### Spin

We encode $\sigma = \uparrow$ by `0` and $\sigma = \downarrow$ by `1`.

In [54]:
# Global parameters

# Distance between two atoms
a = 1

# Number of atoms (in the context of a unit cell with two atoms, the number of unit cells is N/2)
N = 3

# Hopping factor
t = 1

# Length of a side of the crystal assuming square crystal
L = a * N**0.5

# Brillouin zone
wrong -> BZ = [[h,l] for h in range(-int(N**0.5), int(N**0.5) + 1) for l in range(-int(N**0.5), int(N**0.5) + 1) if ((h < 0 or (h == 0 and l < 0)) and abs(h) + abs(l) == N**0.5) or abs(h) + abs(l) < N**0.5]

SyntaxError: invalid syntax (<ipython-input-54-3196befecfa9>, line 16)

In [38]:
def m1_to_nvec(m1):
    nvec = np.zeros(4)
    nvec[0] = (1 + m1) / 2
    nvec[1] = (1 - m1) / 2
    nvec[2] = (1 - m1) / 2
    nvec[3] = (1 + m1) / 2
    return nvec

In [53]:
# The matrix to diagonalize for a given k and spin
def cmat(nvec, k, sigma, U):
    
    h = k[0]
    l = k[1]
    
    gamma_k = - (1 + np.exp(-1.j * 2 * np.pi * h / N**0.5) + np.exp(-1.j * np.pi * (h - l) / N**0.5) + np.exp(-1.j * np.pi * (h + l) / N**0.5))
    
    M = np.zeros((2,2), dtype = complex)
    
    M[0,0] = U * nvec[1 - sigma]
    M[1,1] = U * nvec[3 - sigma]
    
    M[0,1] = t * gamma_k
    M[1,0] = t * np.conj(gamma_k)
    
    return M

In [51]:
# Returns nvec as a function of nvec
def occfunc(nvec, U, T):
    
    new_nvec = np.zeros(4)
    
    # We start by computing all the eigenvectors and eigencoefficients for all k vectors and all spins
    for sigma in [0, 1]:
        for k in BZ:
            
            M = cmat(nvec, k, sigma, U)
            eigvals, eigvecs = np.linalg.eig(M)
            
    new_nvec = 2 * new_nvec / N # N/2 unit cells
            

SyntaxError: invalid syntax (<ipython-input-51-da0824edeea3>, line 6)

In [52]:
# Test

nvec = m1_to_nvec(0.5)
k = BZ[0]
cmats(nvec, k, 0, 1)

array([[0.25      +0.j        , 0.36544249-1.47414136j],
       [0.36544249+1.47414136j, 0.75      +0.j        ]])